In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

In [2]:
from sklearn.pipeline import Pipeline
random_state = 7


In [3]:
from sklearn.impute import SimpleImputer, KNNImputer
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop


In [4]:
path = './AML_data/'
X = joblib.load('./AML_data/meth.pkl')
phenodf = joblib.load('./AML_data/pheno.pkl')


In [5]:
unionindices = joblib.load('./AML_data/unionindices.pkl')
len(unionindices)

1300

In [6]:
phenodf.genotype.unique()

array(['normal', 't(11;19)', nan, 'mono 7', 'inv(16)', 't(9;11)',
       'other clon abn', 'other 11q23/MLL', 't(10;11)', '3q21q26',
       't(8;21)', 'sole+8', 't(15;17)', 'no result'], dtype=object)

In [7]:
len(phenodf.genotype.unique())

14

In [8]:
mll = ['other 11q23/MLL', 't(9;11)', 't(10;11)','t(11;19)']
other = ['normal', 'mono 7', 'inv(16)','other clon abn', '3q21q26', 't(8;21)', 'sole+8', 't(15;17)']

In [9]:
finalgenotype = []

for data in phenodf.genotype:

    if data in mll:
        finalgenotype.append('MLL rearranged')
        
    elif data == 'no result':
        finalgenotype.append('No result')
        
    elif data in other:
        finalgenotype.append(data)
        
    else:
        finalgenotype.append('No result')
        

In [10]:
finalgenotype

['normal',
 'MLL rearranged',
 'normal',
 'No result',
 'mono 7',
 'inv(16)',
 'normal',
 'normal',
 'MLL rearranged',
 'other clon abn',
 'other clon abn',
 'MLL rearranged',
 'normal',
 'MLL rearranged',
 'MLL rearranged',
 'MLL rearranged',
 'other clon abn',
 'MLL rearranged',
 'normal',
 'No result',
 'MLL rearranged',
 'normal',
 'MLL rearranged',
 '3q21q26',
 'normal',
 'other clon abn',
 'other clon abn',
 'inv(16)',
 'No result',
 't(8;21)',
 'sole+8',
 'sole+8',
 'normal',
 'normal',
 'No result',
 'other clon abn',
 'mono 7',
 't(8;21)',
 'normal',
 'inv(16)',
 'No result',
 't(8;21)',
 'MLL rearranged',
 'MLL rearranged',
 'normal',
 'No result',
 'normal',
 't(15;17)',
 'No result',
 'other clon abn',
 'normal',
 'other clon abn',
 'inv(16)',
 'inv(16)',
 't(8;21)',
 'MLL rearranged',
 't(8;21)',
 't(8;21)',
 'normal',
 'normal',
 't(8;21)',
 't(15;17)',
 'No result',
 'MLL rearranged',
 'MLL rearranged',
 'MLL rearranged',
 'inv(16)',
 'No result',
 't(8;21)',
 'No result

In [11]:
phenodf['finalgenotype'] = finalgenotype

In [12]:
phenodf[['genotype', 'finalgenotype']].head(20)

,genotype,finalgenotype
public_id,,
AML_001,normal,normal
AML_002,t(11;19),MLL rearranged
AML_003,normal,normal
AML_004_r,NaN,No result
AML_005,mono 7,mono 7
AML_006,inv(16),inv(16)
AML_007,normal,normal
AML_008,normal,normal
AML_009,t(9;11),MLL rearranged


In [13]:
len(phenodf.finalgenotype.unique())

10

In [14]:
phenodf.finalgenotype.value_counts()

normal            30
MLL rearranged    25
other clon abn    23
No result         20
t(8;21)           19
inv(16)           12
mono 7             5
t(15;17)           4
sole+8             3
3q21q26            1
Name: finalgenotype, dtype: int64

In [15]:
unknowndf = phenodf[phenodf['finalgenotype'].isin(['No result', 'other clon abn'])]

In [16]:
Xun = X[X.index.get_level_values(1).isin(unknowndf.index)]

In [17]:
phenodf = phenodf[~phenodf['finalgenotype'].isin(['No result', 'other clon abn'])]

In [18]:
phenodf.finalgenotype.value_counts()

normal            30
MLL rearranged    25
t(8;21)           19
inv(16)           12
mono 7             5
t(15;17)           4
sole+8             3
3q21q26            1
Name: finalgenotype, dtype: int64

In [19]:
X = X[X.index.get_level_values(1).isin(phenodf.index)]

In [20]:
X.shape

(99, 406830)

In [21]:
Xtrain, Xtest, ytrain, ytest= train_test_split(X, phenodf.finalgenotype, test_size = 0.33, random_state = 6)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(66, 406830) (33, 406830) (66,) (33,)


In [22]:
Xtrain = Xtrain[unionindices]
Xtest = Xtest[unionindices]

In [23]:
Xtrain.shape

(66, 1300)

In [24]:
Xtest.shape

(33, 1300)

In [25]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(Xtrain)
XtrainN = pd.DataFrame(imp.transform(Xtrain), columns = Xtrain.columns, index = Xtrain.index)
XtestN = pd.DataFrame(imp.transform(Xtest), columns = Xtest.columns, index = Xtest.index)


In [26]:
ytrain = pd.DataFrame(ytrain, columns = ['finalgenotype'], index = ytrain.index)
ytest = pd.DataFrame(ytest, columns = ['finalgenotype'], index = ytest.index)


In [27]:
from sklearn.preprocessing import LabelEncoder

In [28]:
encoder = LabelEncoder()
encoder.fit(ytrain)
encoded_ytrain = encoder.transform(ytrain)
encoded_ytest = encoder.transform(ytest)

dummy_ytrain = np_utils.to_categorical(encoded_ytrain)
dummy_yest = np_utils.to_categorical(encoded_ytest)


C:\Users\olgkr928\Anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [29]:
ytrain['classes'] = encoded_ytrain
ytest['classes'] = encoded_ytest

In [30]:
ytrain

,finalgenotype,classes
public_id,,
AML_110,inv(16),2
AML_051,inv(16),2
AML_100,t(8;21),7
AML_015,MLL rearranged,1
AML_123,inv(16),2
...,...,...
AML_047,t(8;21),7
AML_109,mono 7,3
AML_104,normal,4


In [31]:
def baseline_model(input_dim = len(unionindices),
                   out = len(phenodf.finalgenotype.unique()),
                   activation = 'sigmoid',
                   dim1 = 100,
                   dim2 = 50,
                   optimizer = 'rmsprop'):
    model = Sequential()
    model.add(Dense(dim1, input_dim=input_dim, activation=activation)) 
    model.add(Dense(dim2, input_dim=dim1, activation=activation))
    model.add(Dense(out, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [32]:
from keras.models import load_model
# # # Instantiate the model as you please (we are not going to use this)

bestmodel = joblib.load('./AML_data/sklearn_pipelineDLClassification.pkl')
bestmodel.named_steps['NeuralNetwork'].model = load_model('./AML_data/DLSubtype_ClassificationHyperoptAlldata.h5')                      



In [33]:
bestmodel.get_params()

{'memory': None,
 'steps': [('NeuralNetwork',
   <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier at 0x1d494efdca0>)],
 'verbose': False,
 'NeuralNetwork': <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier at 0x1d494efdca0>,
 'NeuralNetwork__verbose': 1,
 'NeuralNetwork__activation': 'sigmoid',
 'NeuralNetwork__batch_size': 8,
 'NeuralNetwork__dim1': 200,
 'NeuralNetwork__epochs': 100,
 'NeuralNetwork__build_fn': <function __main__.baseline_model(input_dim=1300, out=8, activation='sigmoid', dim1=100, dim2=50, optimizer='rmsprop')>}

In [34]:
from sklearn.metrics import accuracy_score, f1_score, plot_confusion_matrix, roc_curve, auc, confusion_matrix, precision_score

# Fit on all training data

In [35]:
bestmodel.fit(XtrainN, dummy_ytrain)

Epoch 1/100
9/9 [==============================] - 0s 2ms/step - loss: 1.8911 - accuracy: 0.1961
Epoch 2/100
9/9 [==============================] - 0s 1ms/step - loss: 1.4691 - accuracy: 0.6208
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 1.3254 - accuracy: 0.6103
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 1.1352 - accuracy: 0.7662
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0684 - accuracy: 0.7727
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9114 - accuracy: 0.7351
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9332 - accuracy: 0.7409
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 0.7313 - accuracy: 0.8125
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5591 - accuracy: 0.8740
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6086 - accuracy: 0.8236
Epoch 11/100
9/9 [===========

Pipeline(steps=[('NeuralNetwork',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x0000017B738F4A00>)])

# Save the model

In [36]:
bm = bestmodel
bm.named_steps['NeuralNetwork'].model.save(path + 'DLSubtype_ClassificationHyperoptAlldataFITTED.h5')
bm.named_steps['NeuralNetwork'].model = None
joblib.dump(bm, path + 'sklearn_pipelineDLClassificationFITTED.pkl')

['./data/outputs/sklearn_pipelineDLClassificationFITTED.pkl']